In [ ]:

import os
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
from scipy.optimize import curve_fit


In [ ]:
# Open datafile
datafile = 'data/AR_12665_133153_0.h5'
print("Reading data file", datafile)
data = h5py.File(datafile, 'r')
print("Opened data file", data.file)

# Print data parameters
print("List of keys in file:", list(data.keys()))
key = list(data.keys())[0]
print("    Number of strokes parameters:   ", data[key].shape[0])
print("    Size of X axis:                 ", data[key].shape[1])
print("    Size of Y axis:                 ", data[key].shape[2])
print("    Number of measured wavelengths: ", data[key].shape[3])

In [ ]:
# Extract each Stokes parameter
i = 0
I = data[key][0]
Q = data[key][1]
U = data[key][2]
V = data[key][3]
stokes_list = [I, Q, U, V]
for stokes in stokes_list:
    print(stokes.shape)
    i = i + 1

In [ ]:
# Plot first image of I data
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 8))
ax.set_xlabel('X axis (array index)')
ax.set_ylabel('Y axis (array index)')
ax.set_title("Intensity data")

img = ax.imshow(I[:,:,0], cmap='gray', origin='lower')
# img = ax.imshow(I[:,:,0], cmap='viridis', vmin=6000, vmax=7500, origin='lower')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax, label='Number of counts')

In [ ]:
# Plot first image for all 4 Stokes parameters
fig, ax = plt.subplots(ncols=4, nrows=1, figsize=(18, 6))
ax[0].set_ylabel('Y axis (array index)')
titles = ['I', 'Q', 'U', 'V']

for i in range(0,4):
    ax[i].set_xlabel('X axis (array index)')
    ax[i].set_title(titles[i])
    img = ax[i].imshow(stokes_list[i][:,:,0], cmap='gray', origin='lower')
    divider = make_axes_locatable(ax[i])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(img, cax=cax)

fig.colorbar(img, cax=cax, label='Number of counts')

In [ ]:
# Open calibration data
calibfile = 'data/fts_calibration.npz'
print("Reading calibration data file", calibfile)
calibdata = np.load(calibfile)
print("Opened calibration data file")

# Print data parameters
print("List of keys in file:", list(calibdata.files))
key = list(calibdata.files)
print("With shape:")
print("    Wavelength daya (x):           ", calibdata[key[0]].shape)
print("    Intensity (y):                 ", calibdata[key[0]].shape)
print("    Continuum (c):                 ", calibdata[key[0]].shape)

In [ ]:
# Select region of the sun for calibration intensity calculation
# sol en calma, sin manchas ni poros --> TODO: como se dice "sol en calma" in English?
xmin = 0
xmax = 220
ymin = 150
ymax = 605
xwidth = xmax - xmin
ywidth = ymax - ymin

# Plot region on image
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 8))
ax.set_xlabel('X axis (array index)')
ax.set_ylabel('Y axis (array index)')
ax.set_title("Intensity data with selected reference region")

img = ax.imshow(I[:,:,0], cmap='gray', origin='lower')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax, label='Number of counts')

# Create a rectangle patch
rect = patches.Rectangle((xmin, ymin), xwidth, ywidth, linewidth=1, edgecolor='r', facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

In [ ]:
# Calculate median intensities to ensure I am doing this right
print("mean of whole image: ", I[:,:,0].mean())
print("mean of patch: ", I[ymin:ymax,xmin:xmax,0].mean())
print("mean of patch in spot: ", I[300:310,400:410,0].mean())
print("mean of patch in penumbra: ", I[400:410,300:310,0].mean())

In [ ]:
# Calculate mean for each wavelength, repeat for each Stokes parameter
mean_stokes_list = np.empty([112,4])

fig, ax = plt.subplots(ncols=1, nrows=4, figsize=(6, 10))
for i in range(4):
    mean_stokes_list[:,i] = stokes_list[i][ymin:ymax,xmin:xmax,:].mean(axis=(0,1))
    
    ax[i].set_ylabel(titles[i])
    img = ax[i].plot(mean_stokes_list[:,i])

In [ ]:
# Get spectral lines from data spectrum
spectrum = mean_stokes_list[:,0]
peaks, _ = find_peaks(-spectrum) 
print("Peaks in I data: ", peaks)

# Example from here: https://eikonaloptics.com/blogs/tutorials/spectrometer-wavelength-calibration-practical-implementation?srsltid=AfmBOoqBsKn0cOmwJ4wTow4yGllnfrRJAqNRn0FOSJ3sFu7leDetbL1D
# Find centroid of spectral lines
npix = 4
centroid_pix = np.array([])
for p in peaks:
  pix = np.arange(p-npix, p+npix+1)
  centroid_pix = np.append(centroid_pix,
                 np.sum(spectrum[p-npix: p+npix+1] * pix) / np.sum(spectrum[p-npix: p+npix+1]))
  
print("Centroix pixels on spectrum: ", centroid_pix)

# Plot on spectrum
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 3))
ax.set_xlabel('Wavelength (not calibrated)')
ax.set_ylabel('Intensity')
ax.set_title("Mean intensity")
ax.vlines(centroid_pix, spectrum.min(), spectrum.max(), colors='orange')
img = ax.plot(spectrum)

In [ ]:
# Plot calibration data
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 3))
ax.set_xlabel('Wavelength')
ax.set_ylabel('Intensity')
ax.set_title("Calibration data")

img = ax.plot(calibdata['x'], calibdata['y'])

In [ ]:
# Get spectral lines from calibration spectrum
calib_spectrum = calibdata['y']
calib_peaks, _ = find_peaks(-calib_spectrum) # find absortion lines
print("Peaks in calibration data: ", calib_peaks)
# Keep peaks which match the Fe I lines (manually)
calib_peaks_clean = [210, 347]

# Get wavelength values of Fe I peaks
calib_wavelengths = [calibdata['x'][210], calibdata['x'][347]]
print("Wavelengths of Fe I peaks: ", calib_wavelengths)

# Plot calibration data
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 3))
ax.set_xlabel('Wavelength [nm]')
ax.set_ylabel('Intensity')
ax.set_title("Calibration data")
ax.vlines(calib_wavelengths, calib_spectrum.min(), calib_spectrum.max(), colors='orange')
img = ax.plot(calibdata['x'], calibdata['y'])

In [ ]:
# Calculate polynomial which fits the spectra to the calibration wavelengths
poly_degree = 1
coeffs_wave_cal = np.polyfit(centroid_pix, calib_wavelengths, deg=poly_degree, w=[20,2])
print("Polyfit coefficients: ", coeffs_wave_cal)

plt.plot(np.polyval(coeffs_wave_cal, np.arange(112)))
plt.title("Polyfit results")

In [ ]:
def fitfunc(x,a,b): # TODO:: finish exploring this option
    return abs(a) * x**2 + b * x

a, b = curve_fit(fitfunc, centroid_pix, calib_wavelengths) 

print(coeffs_wave_cal)
coeffs_wave_cal_2 = [a[0], a[1], 0]
print(coeffs_wave_cal_2)
plt.plot(np.polyval(coeffs_wave_cal_2, np.arange(112)))

In [ ]:
# Adjust spectrum with the calculated polynomial
pix_val = np.arange(len(spectrum))
calibrated_axis = np.polyval(coeffs_wave_cal, pix_val)

# Plot the spectrum with the newly calibrated wavelengths. 
plt.figure(figsize=(8,5))
plt.plot(calibrated_axis, spectrum)
plt.xlabel('Calibrated wavelength [nm]')

In [ ]:
# Normalize spectrum and calibration data, mean of first value
spectrum_n = spectrum / spectrum[0]
calibdata_n = calibdata['y'] / calibdata['y'][0] # this is wrong but useful for visual confirmation

# Compare calibrated spectum to calibration data
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 3))
ax.set_xlabel('Wavelength [nm]')
ax.set_ylabel('Intensity (normalized)')
ax.set_title("Calibration data")
ax.set_xlim([6300, 6304])
img = ax.plot(calibdata['x'], calibdata_n)
img = ax.plot(calibrated_axis, spectrum_n, color='orange')